In [34]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score
from nltk.stem.porter import PorterStemmer
import nltk
from tabulate import tabulate
import re

In [35]:
file_train = pd.read_csv('data_files/train_small.csv')
file_test = pd.read_csv('data_files/test_small.csv')

print(file_train[:5])

  themes process_id                            file_name document_type  pages  \
0  [232]  AI_856934  AI_856934_1926210_1060_17072013.pdf        outros      1   
1  [232]  AI_856934    AI_856934_1926211_34_17072013.pdf        outros      1   
2  [232]  AI_856934    AI_856934_1926211_34_17072013.pdf        outros      2   
3  [232]  AI_856934    AI_856934_1926211_34_17072013.pdf        outros      3   
4  [232]  AI_856934    AI_856934_1926211_34_17072013.pdf        outros      4   

                                                body  
0  {"tribunal justiça estado bahia poder judiciár...  
1  {"excelentíssimo senhor doutor juiz direito ju...  
2  {"razões recurso inominado recorrente atlantic...  
3  {"empresa recorrente tornou credora dos débito...  
4  {"entretanto verdade parte apelante tornou tit...  


In [36]:
x_train = file_train['body']
y_train = file_train['document_type']

x_test = file_test['body']
y_test = file_test['document_type']

print(x_train[0])
print(y_train[0])

{"tribunal justiça estado bahia poder judiciário salvador juizado cível defesa consumidor universo matutino projudi acm campus universidade salgado oliveira universo pituba salvador processo parte autora nailton lantyer cordeiro araujo parte atlantico fundo investimento direitos creditorios despacho vistos etc indefiro requerido pela parte demandante aguarde a sessão conciliação designada salvador de julho paulo alberto nunes chenaud juiz direito documento assinado eletronicamente"}
outros


In [37]:
from gpam_functions import get_stopwords

stops = get_stopwords()

print(stops)

{'ela', 'dela', 'foste', 'algumas', 'sete', 'tenha', 'novo', 'pela', 'devem', 'menos', 'mês', 'conselho', 'quem', 'questão', 'estivéramos', 'naquela', 'onze', 'onde', 'possível', 'tanta', 'veja', 'vêm', 'tem', 'logo', 'nos', 'maiorias', 'estivestes', 'vários', 'ponto', 'nove', 'custa', 'apenas', 'estamos', 'vem', 'daqueles', 'deles', 'porquanto', 'ontem', 'há', 'mil', 'nessas', 'hão', 'quinze', 'pelo', 'dezassete', 'primeiro', 'seu', 'lado', 'tinha', 'através', 'dão', 'nesse', 'querem', 'houverei', 'ainda', 'dos', 'geral', 'primeira', 'vosso', 'dele', 'nível', 'qualquer', 'houvéramos', 'apoia', 'sob', 'tentei', 'teve', 'pode', 'vos', 'inclusive', 'perto', 'sétima', 'usar', 'esta', 'essas', 'final', 'povo', 'houverão', 'teríamos', 'atrás', 'os', 'vossa', 'nosso', 'zero', 'estejam', 'outras', 'favor', 'sexta', 'boa', 'uns', 'relação', 'tinham', 'teriam', 'esteja', 'grupo', 'ora', 'pôde', 'dezasseis', 'está', 'mesmo', 'desta', 'fossem', 'seja', 'lhe', 'fui', 'oitavo', 'deve', 'cuja', 'mom

In [17]:
count_vect = CountVectorizer(stop_words=stops)
x_train_vec = count_vect.fit_transform(x_train)

print(x_train_vec[0])

  (0, 164531)	1
  (0, 91608)	1
  (0, 17373)	1
  (0, 90880)	1
  (0, 148958)	3
  (0, 91025)	1
  (0, 38335)	1
  (0, 40784)	1
  (0, 34175)	1
  (0, 168168)	2
  (0, 104917)	1
  (0, 132787)	1
  (0, 2974)	1
  (0, 24102)	1
  (0, 168156)	1
  (0, 148847)	1
  (0, 118973)	1
  (0, 127862)	1
  (0, 132247)	1
  (0, 16102)	1
  (0, 111322)	1
  (0, 94834)	1
  (0, 35478)	1
  (0, 11433)	1
  (0, 15080)	1
  (0, 67140)	1
  (0, 85064)	1
  (0, 45924)	1
  (0, 36589)	1
  (0, 43604)	1
  (0, 172997)	1
  (0, 58292)	1
  (0, 81863)	1
  (0, 142424)	1
  (0, 41480)	1
  (0, 5891)	1
  (0, 152062)	1
  (0, 32352)	1
  (0, 43282)	1
  (0, 91105)	1
  (0, 124568)	1
  (0, 6764)	1
  (0, 115971)	1
  (0, 28006)	1
  (0, 91021)	1
  (0, 45921)	1
  (0, 47576)	1
  (0, 14326)	1
  (0, 52509)	1


In [19]:
count_vect.get_feature_names_out()[148958]

'salvador'

In [38]:
from gpam_functions import remove_stop_words

data = list(x_train)
print(x_train[0])
print(data[0])
i = 0
for item in data:
  data[i] = remove_stop_words(item)
  i += 1

x_train = pd.Series(data)

{"tribunal justiça estado bahia poder judiciário salvador juizado cível defesa consumidor universo matutino projudi acm campus universidade salgado oliveira universo pituba salvador processo parte autora nailton lantyer cordeiro araujo parte atlantico fundo investimento direitos creditorios despacho vistos etc indefiro requerido pela parte demandante aguarde a sessão conciliação designada salvador de julho paulo alberto nunes chenaud juiz direito documento assinado eletronicamente"}
{"tribunal justiça estado bahia poder judiciário salvador juizado cível defesa consumidor universo matutino projudi acm campus universidade salgado oliveira universo pituba salvador processo parte autora nailton lantyer cordeiro araujo parte atlantico fundo investimento direitos creditorios despacho vistos etc indefiro requerido pela parte demandante aguarde a sessão conciliação designada salvador de julho paulo alberto nunes chenaud juiz direito documento assinado eletronicamente"}


In [44]:
data_test = list(x_test)
print(x_test[0])
print(data_test[0])
i = 0
for item in data_test:
  data_test[i] = remove_stop_words(item)
  i += 1

x_test = pd.Series(data_test)

{"processo termo remessa tendo vista disposto portaria dje considerando decidido re remeto presentes autos origem brasília de julho luciana pires zavala secretária judiciária supremo tribunal federal lavrei este termo"}
{"processo termo remessa tendo vista disposto portaria dje considerando decidido re remeto presentes autos origem brasília de julho luciana pires zavala secretária judiciária supremo tribunal federal lavrei este termo"}


In [45]:
print(x_test[:5])

0    {"processo termo remessa tendo vista disposto ...
1    {"supremo tribunal federal ofício brasília  ou...
2    {"excelentíssimo senhor juiz presidente turma ...
3    {"advogados agravado vitor rodrigues moura oab...
4    {"eminentes ministros tempestividade publicaçã...
dtype: object


In [40]:
print(x_train[:5])

0    {"tribunal justiça  bahia  judiciário salvador...
1    {"excelentíssimo senhor doutor juiz direito ju...
2    {"razões recurso inominado recorrente atlantic...
3    {"empresa recorrente tornou credora  débitos n...
4    {"entretanto verdade  apelante tornou titular ...
dtype: object


In [42]:
pipeline = Pipeline([
  ('vect', CountVectorizer()),
  ('clf', SGDClassifier()),
])

parameters = {
  'vect__max_df': ([0.5, 0.75]),
  'vect__ngram_range': ([(1, 1), (1, 2)]), # unigrams or bigrams
  'clf__alpha': ([1.0, 0.01]),
}

scoring = {
  'accuracy',
  'f1_micro',
  'f1_macro',
  'f1_weighted'
}

grid_search = GridSearchCV(estimator=pipeline, param_grid=parameters, n_jobs=None, verbose=3, return_train_score=True, scoring=scoring, refit='f1_micro')

grid_search.fit(x_train, y_train)

print('Best parameters: {}'.format(grid_search.best_params_))
print('Best score: {}'.format(grid_search.best_score_))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END clf__alpha=1.0, vect__max_df=0.5, vect__ngram_range=(1, 1); accuracy: (train=0.899, test=0.899) f1_macro: (train=0.158, test=0.158) f1_micro: (train=0.899, test=0.899) f1_weighted: (train=0.851, test=0.851) total time=  21.7s
[CV 2/5] END clf__alpha=1.0, vect__max_df=0.5, vect__ngram_range=(1, 1); accuracy: (train=0.899, test=0.899) f1_macro: (train=0.158, test=0.158) f1_micro: (train=0.899, test=0.899) f1_weighted: (train=0.851, test=0.851) total time=  22.5s
[CV 3/5] END clf__alpha=1.0, vect__max_df=0.5, vect__ngram_range=(1, 1); accuracy: (train=0.899, test=0.899) f1_macro: (train=0.158, test=0.158) f1_micro: (train=0.899, test=0.899) f1_weighted: (train=0.851, test=0.851) total time=  22.1s
[CV 4/5] END clf__alpha=1.0, vect__max_df=0.5, vect__ngram_range=(1, 1); accuracy: (train=0.899, test=0.899) f1_macro: (train=0.158, test=0.158) f1_micro: (train=0.899, test=0.899) f1_weighted: (train=0.851, test=0.851) tot

In [46]:
best_estimator = grid_search.best_estimator_

print('F1 Score (micro): {}'.format(f1_score(y_test, best_estimator.predict(x_test), average='micro')))

F1 Score (micro): 0.9373364319661662


In [47]:
pipeline = Pipeline([
  ('vect', TfidfVectorizer()),
  ('clf', MultinomialNB()),
])

parameters = {
  'vect__max_df': ([0.5, 0.75]),
  'vect__ngram_range': ([(1, 1), (1, 2)]), # unigrams or bigrams
  'clf__alpha': ([1.0, 0.01]),
}

scoring = {
  'accuracy',
  'f1_micro',
  'f1_macro',
  'f1_weighted'
}

grid_search = GridSearchCV(estimator=pipeline, param_grid=parameters, n_jobs=None, verbose=3, return_train_score=True, scoring=scoring, refit='f1_micro')

grid_search.fit(x_train, y_train)

print('Best parameters: {}'.format(grid_search.best_params_))
print('Best score: {}'.format(grid_search.best_score_))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END clf__alpha=1.0, vect__max_df=0.5, vect__ngram_range=(1, 1); accuracy: (train=0.893, test=0.898) f1_macro: (train=0.176, test=0.168) f1_micro: (train=0.893, test=0.898) f1_weighted: (train=0.854, test=0.854) total time=  16.0s
[CV 2/5] END clf__alpha=1.0, vect__max_df=0.5, vect__ngram_range=(1, 1); accuracy: (train=0.891, test=0.897) f1_macro: (train=0.165, test=0.170) f1_micro: (train=0.891, test=0.897) f1_weighted: (train=0.850, test=0.855) total time=  19.1s
[CV 3/5] END clf__alpha=1.0, vect__max_df=0.5, vect__ngram_range=(1, 1); accuracy: (train=0.892, test=0.897) f1_macro: (train=0.171, test=0.166) f1_micro: (train=0.892, test=0.897) f1_weighted: (train=0.852, test=0.853) total time=  19.3s
[CV 4/5] END clf__alpha=1.0, vect__max_df=0.5, vect__ngram_range=(1, 1); accuracy: (train=0.894, test=0.898) f1_macro: (train=0.183, test=0.159) f1_micro: (train=0.894, test=0.898) f1_weighted: (train=0.858, test=0.851) tot

In [48]:
best_estimator = grid_search.best_estimator_

print('F1 Score (micro): {}'.format(f1_score(y_test, best_estimator.predict(x_test), average='micro')))

F1 Score (micro): 0.9055754454284697


In [49]:
pipeline = Pipeline([
  ('vect', CountVectorizer()),
  ('clf', MultinomialNB()),
])

parameters = {
  'vect__max_df': ([0.5, 0.75]),
  'vect__ngram_range': ([(1, 1), (1, 2)]), # unigrams or bigrams
  'clf__alpha': ([1.0, 0.01]),
}

scoring = {
  'accuracy',
  'f1_micro',
  'f1_macro',
  'f1_weighted'
}

grid_search = GridSearchCV(estimator=pipeline, param_grid=parameters, n_jobs=None, verbose=3, return_train_score=True, scoring=scoring, refit='f1_micro')

grid_search.fit(x_train, y_train)

print('Best parameters: {}'.format(grid_search.best_params_))
print('Best score: {}'.format(grid_search.best_score_))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END clf__alpha=1.0, vect__max_df=0.5, vect__ngram_range=(1, 1); accuracy: (train=0.810, test=0.828) f1_macro: (train=0.483, test=0.360) f1_micro: (train=0.810, test=0.828) f1_weighted: (train=0.844, test=0.855) total time=  16.6s
[CV 2/5] END clf__alpha=1.0, vect__max_df=0.5, vect__ngram_range=(1, 1); accuracy: (train=0.789, test=0.693) f1_macro: (train=0.491, test=0.434) f1_micro: (train=0.789, test=0.693) f1_weighted: (train=0.830, test=0.759) total time=  16.9s
[CV 3/5] END clf__alpha=1.0, vect__max_df=0.5, vect__ngram_range=(1, 1); accuracy: (train=0.771, test=0.703) f1_macro: (train=0.465, test=0.432) f1_micro: (train=0.771, test=0.703) f1_weighted: (train=0.817, test=0.766) total time=  17.2s
[CV 4/5] END clf__alpha=1.0, vect__max_df=0.5, vect__ngram_range=(1, 1); accuracy: (train=0.801, test=0.761) f1_macro: (train=0.478, test=0.427) f1_micro: (train=0.801, test=0.761) f1_weighted: (train=0.838, test=0.802) tot

In [50]:
best_estimator = grid_search.best_estimator_

print('F1 Score (micro): {}'.format(f1_score(y_test, best_estimator.predict(x_test), average='micro')))

F1 Score (micro): 0.9002889265749638


In [51]:
pipeline = Pipeline([
  ('vect', TfidfVectorizer()),
  ('clf', SGDClassifier()),
])

parameters = {
  'vect__max_df': ([0.5, 0.75]),
  'vect__ngram_range': ([(1, 1), (1, 2)]), # unigrams or bigrams
  'clf__alpha': ([1.0, 0.01]),
}

scoring = {
  'accuracy',
  'f1_micro',
  'f1_macro',
  'f1_weighted'
}

grid_search = GridSearchCV(estimator=pipeline, param_grid=parameters, n_jobs=None, verbose=3, return_train_score=True, scoring=scoring, refit='f1_micro')

grid_search.fit(x_train, y_train)

print('Best parameters: {}'.format(grid_search.best_params_))
print('Best score: {}'.format(grid_search.best_score_))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END clf__alpha=1.0, vect__max_df=0.5, vect__ngram_range=(1, 1); accuracy: (train=0.899, test=0.899) f1_macro: (train=0.158, test=0.158) f1_micro: (train=0.899, test=0.899) f1_weighted: (train=0.851, test=0.851) total time=  24.3s
[CV 2/5] END clf__alpha=1.0, vect__max_df=0.5, vect__ngram_range=(1, 1); accuracy: (train=0.899, test=0.899) f1_macro: (train=0.158, test=0.158) f1_micro: (train=0.899, test=0.899) f1_weighted: (train=0.851, test=0.851) total time=  23.3s
[CV 3/5] END clf__alpha=1.0, vect__max_df=0.5, vect__ngram_range=(1, 1); accuracy: (train=0.899, test=0.899) f1_macro: (train=0.158, test=0.158) f1_micro: (train=0.899, test=0.899) f1_weighted: (train=0.851, test=0.851) total time=  23.6s
[CV 4/5] END clf__alpha=1.0, vect__max_df=0.5, vect__ngram_range=(1, 1); accuracy: (train=0.899, test=0.899) f1_macro: (train=0.158, test=0.158) f1_micro: (train=0.899, test=0.899) f1_weighted: (train=0.851, test=0.851) tot

In [52]:
best_estimator = grid_search.best_estimator_

print('F1 Score (micro): {}'.format(f1_score(y_test, best_estimator.predict(x_test), average='micro')))

F1 Score (micro): 0.8940811925549065


In [53]:
header = ['Classifiers', 'vect__max_df', 'vect__ngram_range', 'clf__alpha', 'Best Score', 'F1 Score - micro']

metrics = []

metrics.append(['CountVectorizer + SGDClassifier', '0.5', '(1, 2)', '0.01', '0.919600', '0.937336'])
metrics.append(['TfidfVectorizer + MultinomialNB', '0.75', '(1, 2)', '1.0', '0.902826', '0.905575'])
metrics.append(['CountVectorizer + MultinomialNB', '0.75', '(1, 2)', '1.0', '0.863353', '0.900288'])
metrics.append(['TfidfVectorizer + SGDClassifier', '0.5', '(1, 1)', '1.0', '0.898919', '0.894081'])

print(tabulate(metrics, headers=header, tablefmt="fancy_grid"))

╒═════════════════════════════════╤════════════════╤═════════════════════╤══════════════╤══════════════╤════════════════════╕
│ Classifiers                     │   vect__max_df │ vect__ngram_range   │   clf__alpha │   Best Score │   F1 Score - micro │
╞═════════════════════════════════╪════════════════╪═════════════════════╪══════════════╪══════════════╪════════════════════╡
│ CountVectorizer + SGDClassifier │           0.5  │ (1, 2)              │         0.01 │     0.9196   │           0.937336 │
├─────────────────────────────────┼────────────────┼─────────────────────┼──────────────┼──────────────┼────────────────────┤
│ TfidfVectorizer + MultinomialNB │           0.75 │ (1, 2)              │         1    │     0.902826 │           0.905575 │
├─────────────────────────────────┼────────────────┼─────────────────────┼──────────────┼──────────────┼────────────────────┤
│ CountVectorizer + MultinomialNB │           0.75 │ (1, 2)              │         1    │     0.863353 │           0.9